In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
dataset,info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)
train_dataset,test_dataset = dataset['train'],dataset['test']
train_dataset.element_spec
for example,label in train_dataset.take(1):
    print('text:',example.numpy())
    print('label:',label.numpy())
    BUFFER_SIZE=10000
    BATCH_SIZE=64
    train_dataset=train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    test_dataset=test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    VOCAB_SIZE=1000
    encoder=tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
    encoder.adapt(train_dataset.map(lambda text,label:text))
    model=tf.keras.Sequential([encoder,tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()),output_dim=64,mask_zero=True),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                              tf.keras.layers.Dense(64,activation='relu'),
                              tf.keras.layers.Dense(1)
                              ])
    sample_text=('The movie was cool. The animation and the graphics'
                'were out of this world. I would recommend this movie')
    predictions = model.predict(np.array([sample_text]))
    print(predictions[0])
    print("$$$$$$$$$$$$$$$")
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(1e-4),
                 metrics=['accuracy'])
    history = model.fit(train_dataset, epochs=2,
                       validation_data=test_dataset,
                       validation_steps=30)
    test_loss,test_acc=model.evaluate(test_dataset)
    print('Test Los:',test_loss)
    print('Test Accuracy:',test_acc)

text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label: 0
1/1 [==============================] - 1s 1s/step
[-0.00625011]
$$$$$$$$$$$$$$$
Epoch 1/2
391/391 [==============================] - 607s 2s/step - loss: 0.6507 - accuracy: 0.5566 - val_loss: 0.5069 - val_accuracy: 0.7583
Epoch 2/2
391/391 [==============================]